# Decision Tree

In [1]:
from math import log
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt

# 1. Information Entropy
## 1.1 Definition
Entrophy is a measure of the amount of uncertainty in the dataset.
$$H(X) = -\sum_{x\in X} p(x) \log_2 p(x)$$

S: The current dataset for which entropy is being calculated  
X: The set of classes in S  
p(x): The proportion of the number of elements in class x to the number of elements in set S  

Source: [wikipedia](https://en.wikipedia.org/wiki/ID3_algorithm#
)

In [2]:
def entropy(dataset):
    H = 0
    m = len(dataset)
    label_dic = {}
    for data in dataset:
        label = data[-1]
        if label not in label_dic:
            label_dic[label] = 0
        label_dic[label] += 1
    for key, value in label_dic.items():
        p = value/m
        H -= p*log(p,2)
    return H

In [3]:
test_data = [[1,1,'yes'],[1,1,'yes'],[1,0,'no'],[0,1,'no'],[0,1,'no']]
print(entropy(test_data))
test_data[0][-1] = 'maybe'
print(entropy(test_data)) #larger H means greater chaos

0.9709505944546686
1.3709505944546687


## 1.2 Dataset Split
Select records whose features value meet the requirement. The selected feature will not be included in the returned dataset.

In [4]:
def split(dataset, feature, value):
    '''dataset[i]: [feature0, feature1, ..., label],
    extract records with record[feature] == value'''
    split_dataset = []
    for record in dataset:
        if record[feature] == value:
            split_record = record[:feature]
            split_record.extend(record[(feature+1):])
            split_dataset.append(split_record)
    return split_dataset

In [5]:
test_data = [[1,1,'yes'],[1,1,'yes'],[1,0,'no'],[0,1,'no'],[0,1,'no']]
print(split(test_data, 0,1))
print(split(test_data, 0,0))

[[1, 'yes'], [1, 'yes'], [0, 'no']]
[[1, 'no'], [1, 'no']]


## 1.3 Choose a Feature

### 1.3.1 Conditional Information Entropy and Information Gain
The feature which leads to the greatest information gain should be selected.
$$\begin{align*}
H(X|Y) &= \sum_{y \in Y} H(X|Y=y)\\
&= -\sum_{y \in Y}p(y) \sum_{X=x}p(x|y)\log{p(x|y)}\\
&= -\sum_{y \in Y}\sum_{X=x}p(x,y)\log{p(x|y)}\\
G(X,Y) &= H(X) - H(X|Y)
\end{align*}$$

### 1.3.2 Code Implemetation

In [6]:
def choose_feature(dataset):
    num = len(dataset[0]) - 1
    HX = entropy(dataset)
    gain = 0
    feature_index = 0
    for i in range(num):
        HXY = 0
        vector = [record[i] for record in dataset]
        vector_set = set(vector) #find all unique values of a feature
        for element in vector_set:
            subdataset = split(dataset, i, element)
            Hy = entropy(subdataset)
            HXY += (len(subdataset)/len(dataset))*Hy #weighed by prob of each value
        if HX - HXY > gain:
            feature_index = i
            gain = HX - HXY
    return feature_index        

In [7]:
test_data = [[1,1,'yes'],[1,1,'yes'],[1,0,'no'],[0,1,'no'],[0,1,'no']]
choose_feature(test_data)

0

## 2. Tree Construction by Recursion 
For each value of selected feature, repeat the algorithm to determine the sub-tree. Stoping criteria would be:  
1. All records are with the same label.
2. Records are not with the same label while running out of features, use the most common label to mark this leaf.

In [8]:
def ID3(dataset, feature_name):
    '''Use a dict to record structure of a tree'''
    labels = [record[-1] for record in dataset]
    if labels.count(labels[0]) == len(labels): #all records are with the same label
        return labels[0]
    if len(dataset[0]) == 1:
        return Counter(labels).most_common(1)[0][0] #if all features have been traversed, return the most popular label
    
    best_feature = choose_feature(dataset)
    best_feature_name = feature_name[best_feature]
    tree = {best_feature_name:{}}
    feature_name = feature_name[:best_feature] + feature_name[best_feature+1:]
    
    vector = [record[best_feature] for record in dataset]
    vector_set = set(vector)
    for value in vector_set:
        tree[best_feature_name][value] = ID3(split(dataset, best_feature,value),feature_name)
    return tree

In [9]:
test_data = [[1,1,'yes'],[1,1,'yes'],[1,0,'no'],[0,1,'no'],[0,1,'no']]
ID3(test_data, ['no surfacing','flippers'])

{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}

## 3. Plot the Structure of Tree
To be compeleted

## 4. Use a Decision Tree for Classification
Assume now we have a tree, although we are quite clear about its structure, we may not be sure about the location of the node in the dataset. For example, a node is named as 'no surfacing', how could we tell whether the feature 'no surfacing' is at the first column, or second in the dataset? This problem could be solved by recursion for each node in a tree.

In [10]:
def classify(tree, labels, subject):
    firstkey = list(tree.keys())[0]
    subdic = tree[firstkey]
    feature = labels.index(firstkey)
    for key in subdic.keys():
        if subject[feature] == key:
            if type(subdic[key]).__name__ == 'dict':
                label = classify(subdic[key], labels, subject)
            else:
                label = subdic[key]
    return label

In [11]:
tree = {'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}
print(classify(tree, ['no surfacing','flippers'], [1,0]))
print(classify(tree, ['no surfacing','flippers'], [1,1]))

no
yes


## 5. Lense Suggestion

In [12]:
lense_data = []
with open('lenses.txt') as f:
    content = f.readlines()
    for line in content:
        line = line.strip()
        line = line.split('\t')
        lense_data.append(line)
        
lense_data[:5][:]

[['young', 'myope', 'no', 'reduced', 'no lenses'],
 ['young', 'myope', 'no', 'normal', 'soft'],
 ['young', 'myope', 'yes', 'reduced', 'no lenses'],
 ['young', 'myope', 'yes', 'normal', 'hard'],
 ['young', 'hyper', 'no', 'reduced', 'no lenses']]

In [13]:
lense_labels = ['age','prescript','astigmatic','tearrate']
lense_tree = ID3(lense_data, lense_labels)
lense_tree

{'tearrate': {'reduced': 'no lenses',
  'normal': {'astigmatic': {'yes': {'prescript': {'hyper': {'age': {'presbyopic': 'no lenses',
        'young': 'hard',
        'pre': 'no lenses'}},
      'myope': 'hard'}},
    'no': {'age': {'presbyopic': {'prescript': {'hyper': 'soft',
        'myope': 'no lenses'}},
      'young': 'soft',
      'pre': 'soft'}}}}}}